# Query cellxgene-census using TileDB-SOMA

The [first guide](cellxgene) queried metadata and h5ad artifacts directly through LaminDB.

This guide uses the TileDB-SOMA API to run similar queries.

## Setup

Load your LaminDB instance for quering data:

In [ ]:
!lamin load laminlabs/cellxgene

In [ ]:
import lamindb as ln
import bionty as bt
import tiledbsoma

census_version = "2023-12-15"

## Create lookup objects

We use metadata records in the `laminlabs/cellxgene` instance to generate lookups:

In [ ]:
human = "homo_sapiens"

features = ln.Feature.lookup(return_field="name")
assays = bt.ExperimentalFactor.lookup(return_field="name")
cell_types = bt.CellType.lookup(return_field="name")
tissues = bt.Tissue.lookup(return_field="name")
ulabels = ln.ULabel.lookup()
suspension_types = ulabels.is_suspension_type.children.all().lookup(return_field="name")

## Query data

In [ ]:
value_filter = (
    f'{features.tissue} == "{tissues.brain}" and {features.cell_type} in'
    f' ["{cell_types.microglial_cell}", "{cell_types.neuron}"] and'
    f' {features.suspension_type} == "{suspension_types.cell}" and {features.assay} =='
    f' "{assays.ln_10x_3_v3}"'
)

In [ ]:
value_filter

In [ ]:
census_artifact = ln.Artifact.filter(description="Census "+census_version).one()

In [ ]:
census_path = census_artifact.path

region = census_path.fs.call_s3(
    "head_bucket", 
    Bucket=census_path.drive
)["ResponseMetadata"]["HTTPHeaders"]["x-amz-bucket-region"]

In [ ]:
ctx = tiledbsoma.SOMATileDBContext(tiledb_config={"vfs.s3.region": region})

In [ ]:
with tiledbsoma.Collection.open(census_path.as_posix(), context=ctx) as census:
    # Reads SOMADataFrame as a slice
    cell_metadata = census["census_data"][human].obs.read(value_filter=value_filter)

    # Concatenates results to pyarrow.Table
    cell_metadata = cell_metadata.concat()

    # Converts to pandas.DataFrame
    cell_metadata = cell_metadata.to_pandas()

In [ ]:
cell_metadata.shape

In [ ]:
cell_metadata.head()

## Create AnnData

In [ ]:
with tiledbsoma.Collection.open(census_path.as_posix(), context=ctx) as census:
    
    experiment = census["census_data"][human]
    
    adata = experiment.axis_query(
        "RNA",
        obs_query=tiledbsoma.AxisQuery(value_filter=value_filter)
    ).to_anndata(
        X_name="raw",
        column_names={
            "obs": [
                features.assay,
                features.cell_type,
                features.tissue,
                features.disease,
                features.suspension_type,
            ]
        }
    )

In [ ]:
adata.var = adata.var.set_index("feature_id")

In [ ]:
adata

In [ ]:
adata.var.head()

In [ ]:
adata.obs.head()